<a href="https://colab.research.google.com/github/FabioCerqueiraGit/DataSciencePython/blob/main/FraudulentCreditCard/DTS_FraudeCartao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Definição do Problema

Objetivo: Identificar características e padrões em transações fraudulentas no dataset fornecido.

# 2. Coleta e Preparação dos Dados

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Carregar os dados
df = pd.read_csv('/content/creditcard.csv')

# Mostrar as primeiras linhas do dataset
df.head()

# Verificar se há dados ausentes
df.isnull().sum()


# 3. Análise Descritiva

In [ ]:
# Estatísticas descritivas
df.describe()

# Distribuição das classes
plt.figure(figsize=(6, 4))
sns.countplot(x='Class', data=df)
plt.title('Distribuição de Classes')
plt.show()

# Distribuição das transações pelo tempo
plt.figure(figsize=(10, 6))
sns.histplot(df['Time'], bins=50, kde=True)
plt.title('Distribuição das Transações pelo Tempo')
plt.show()

# Distribuição dos valores das transações
plt.figure(figsize=(10, 6))
sns.histplot(df['Amount'], bins=50, kde=True)
plt.title('Distribuição dos Valores das Transações')
plt.show()


# 4. Análise Diagnóstica

In [ ]:
# Correlação entre as variáveis
plt.figure(figsize=(15, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

# Análise das variáveis PCA
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0]

plt.figure(figsize=(15, 10))
for i in range(1, 29):
    plt.subplot(7, 4, i)
    sns.kdeplot(fraud[f'V{i}'], label='Fraude', shade=True)
    sns.kdeplot(non_fraud[f'V{i}'], label='Não Fraude', shade=True)
    plt.title(f'V{i}')
plt.tight_layout()
plt.show()


# 5. Análise Preditiva

In [ ]:
# Preparação dos dados para modelagem
X = df.drop('Class', axis=1)
y = df['Class']

# Divisão em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Treinamento do modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Avaliação do modelo
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title('Matriz de Confusão')
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.show()

# Curva ROC
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()


# 6. Análise Prescritiva

In [ ]:
# Análise dos recursos mais importantes do modelo
feature_importances = model.feature_importances_
features = X.columns
importances = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importances = importances.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=importances)
plt.title('Importância das Variáveis')
plt.show()

# Recomendações baseadas nas análises
print("Recomendações para minimizar fraudes:")
print("- Monitorar as transações com altos valores de variáveis importantes (ex: V4, V11, V12, etc.).")
print("- Implementar verificações adicionais para transações fora do padrão normal de comportamento.")
print("- Usar o modelo de machine learning em tempo real para detectar e bloquear possíveis fraudes.")


# Considerações Finais
Descritiva: Exploramos a distribuição das classes e as características gerais das transações.

Diagnóstica: Investigamos correlações e padrões nas variáveis que diferenciam transações fraudulentas das não fraudulentas.

Preditiva: Treinamos um modelo de Random Forest para prever fraudes e avaliamos seu desempenho.

Prescritiva: Analisamos as variáveis mais importantes no modelo e propusemos ações para minimizar fraudes.